<b><h1 align="center">Data Preparation for BI</h1></b>
<b><h1 align="center">Preparación de datos para Inteligencia de Negocio</h1></b>
</i>



---

Como parte de la etapa final de este proyecto ser creará un tablero de Power BI para hacer semiento de las métricas e indicadores más importantes para el modelo de negocio. Para lograr extraer conocimiento realmente útil en necesario hacer cierto tratamiento a los datos disponibles. Si bien es cierto que este tratamiento puede llevarse a cabo directamente, hasta cierto punto, dentro de la herramienta de Business Intelligence, se opta por crear un cuaderno específico para dicho tratamiento. 

Esto tiene con ventaja que se sigue el mismo ambiente para el procesamiento de datos utilizados para la extración, transformación y carga de datos del modelo predictivo.

## Cargas iniciales

In [155]:
import numpy as np
import pandas as pd
from datetime import date

In [156]:
# Cargar datos
df = pd.read_hdf("Data/data.h5", key="ventas", mode="r")
df = df.set_index("Fec. Fact. Venta")

In [157]:
df["Cliente"].head()

Fec. Fact. Venta
2010-01-08                              CDV CONSTRUCTORA LTDA. 
2010-01-08    RICARDO ANTONIO CONCHA SILVA ARQ. Y SERV. INMO...
2010-01-08      VIDEOCORP INGENIERIA Y TELECOMUNICACIONES S.A. 
2010-01-08      VIDEOCORP INGENIERIA Y TELECOMUNICACIONES S.A. 
2010-01-08      VIDEOCORP INGENIERIA Y TELECOMUNICACIONES S.A. 
Name: Cliente, dtype: object

## Indentificación de clientes 80/20

Tal como su nombre lo indica, los clinetes 80/20 son aquellos clientes responsable por el 80% de las ventas (aproximadamente) y que suelen representar el 20% de la lista total de clientes (aproximadamente). No todos los años cada cliente compra la misma cantidad. Por ello se utiliza una histórico de los últimos 3 años para identificar a los clientes pertenecientes de esta categoría.

In [158]:
# Últimos 3 años
last_years = df.loc[date(2019,1,1):date(2021,1,1)]

by_client = last_years.groupby("Cliente").sum()
by_client["Valor"].head()

Cliente
2D INGENIEROS CONSULTORES LIMITADA               6599256.0
303 EDITORIALES S.A.                             1391197.0
360 PROYECTO DE INGENIERÍA LIMITADA              1495361.0
3C PROPIEDADES GESTION INMOBILIARIA LIMITADA       73800.0
3L INGENIERIA SPA                                 822800.0
Name: Valor, dtype: float64

In [159]:
# Clientes 80/20
total_sell = by_client["Valor"].sum()
clients = by_client["Valor"].sort_values(ascending=False).index
sell_cum = 0
main_clients = []
for i, val in enumerate(by_client["Valor"].sort_values(ascending=False)):
    # Sumar hasta que se alcance el 80% de la venta
    if sell_cum / total_sell < 0.8:
        sell_cum += val
        main_clients.append(clients[i])
        
print(f"Los clientes responsables del 80% de las ventas son {len(main_clients)}.")
print(f"Estos representan un {round(100*len(main_clients)/len(clients),1)}% del total de clientes.") 

Los clientes responsables del 80% de las ventas son 328.
Estos representan un 16.1% del total de clientes.


In [160]:
# Guardemos esta etiqueta en la base de datos
df["80/20"] = [True if client in main_clients else False for client in df["Cliente"]]

## Renovaciones

In [161]:
df["New/Renew"] = ["Renew" if not desc.find("Renew") == -1
                   else "Renew" if not desc.find("RENOVA") == -1
                   else "Renew" if not desc.find("RENEW") == -1
                   else "New"
                   for desc in df["Descripción"]]

## Línea de producto

In [162]:
df["Línea"] = ["Autodesk" if not group.find("AUTODESK") == -1
               else "Consultoría" if not group.find("CONSULTOR") == -1
               else "Cursos" if not group.find("CURSO") == -1
               else "Otros"
               for group in df["Grupo"]]

In [163]:
df["Línea"] = ["Consultoría" if not desc.find("CONSULTOR")
               else df["Línea"].iloc[i]
               for i, desc in enumerate(df["Descripción"])]

In [164]:
df.to_csv("Data/bi_data.csv")